In [11]:
import pandas as pd

def anidar_datos_csv_sin_duplicados(casinos_file, geocoded_file, output_file):
    """
    Combina datos de casinos y resultados geocodificados, manejando problemas de codificación,
    eliminando filas duplicadas en el resultado final y guardando delimitado por punto y coma.

    Args:
        casinos_file (str): Ruta al archivo CSV de datos de casinos (delimitado por comas).
        geocoded_file (str): Ruta al archivo CSV de resultados geocodificados (delimitado por punto y coma).
        output_file (str): Nombre del archivo CSV de salida con los datos combinados.
    """
    try:
        # Cargar el primer CSV (datos_casinos_salas.csv)
        # Recomiendo 'latin1' si 'utf-8' te dio problemas con los caracteres especiales antes.
        # Si 'utf-8' FUNCIONA para los caracteres, déjalo así.
        df_casinos = pd.read_csv(casinos_file, delimiter=',', encoding='utf-8')
        print(f"'{casinos_file}' cargado exitosamente con codificación 'utf-8'.")
        print(f"Columnas en '{casinos_file}': {df_casinos.columns.tolist()}")

        # Cargar el segundo CSV (resultados_geocodificados.csv)
        # Igual, si 'utf-8' funciona para este archivo, déjalo así.
        # Si no, cambia a 'latin1'.
        df_geocoded = pd.read_csv(geocoded_file, delimiter=';', encoding='utf-8')
        print(f"'{geocoded_file}' cargado exitosamente con codificación 'utf-8'.")
        print(f"Columnas en '{geocoded_file}': {df_geocoded.columns.tolist()}")

        # **Paso crucial para evitar duplicados en la unión:**
        # Asegurarse de que el DataFrame de geocodificación tenga direcciones únicas para la unión.
        # Si hay varias entradas para la misma 'Direccion_Original' en 'resultados_geocodificados.csv',
        # el merge podría crear duplicados o elegir una arbitrariamente.
        # Aquí, tomamos la primera ocurrencia si hay direcciones duplicadas en el archivo geocodificado.
        df_geocoded = df_geocoded.drop_duplicates(subset=['Direccion_Original'], keep='first')
        print(f"Eliminadas direcciones duplicadas en '{geocoded_file}' para la columna 'Direccion_Original'.")


        # Renombrar la columna 'Direccion_Original' en df_geocoded para que coincida con 'Dirección' en df_casinos
        df_geocoded = df_geocoded.rename(columns={'Direccion_Original': 'Dirección'})

        # Realizar la combinación (anidación) de los dos DataFrames
        df_combinado = pd.merge(df_casinos, df_geocoded, on='Dirección', how='left')

        # **Eliminar filas completamente duplicadas en el DataFrame resultante.**
        # Esto eliminará cualquier fila donde *todos* los valores de las columnas sean idénticos a otra fila.
        filas_antes = len(df_combinado)
        df_combinado = df_combinado.drop_duplicates()
        filas_despues = len(df_combinado)
        print(f"Se eliminaron {filas_antes - filas_despues} filas completamente duplicadas después de la combinación.")


        # Guardar el DataFrame combinado en un nuevo archivo CSV
        # Especificamos el delimitador ';' para el archivo de salida
        # Usamos 'latin1' como codificación para la escritura para preservar los caracteres especiales.
        df_combinado.to_csv(output_file, index=False, sep=';', encoding='latin1')
        print(f"\nDatos combinados guardados exitosamente en '{output_file}' delimitado por punto y coma y codificación 'latin1'.")

    except FileNotFoundError:
        print(f"Error: Uno de los archivos no fue encontrado. Asegúrate de que '{casinos_file}' y '{geocoded_file}' estén en la misma carpeta que el script.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

In [12]:
# Nombres de tus archivos CSV
casinos_csv = 'datos_casinos_salas.csv'
geocoded_csv = 'resultados_geocodificados.csv'
output_csv = 'datos_casinos_geocodificados.csv'

# Llamar a la función para anidar los datos
anidar_datos_csv_sin_duplicados(casinos_csv, geocoded_csv, output_csv)

'datos_casinos_salas.csv' cargado exitosamente con codificación 'utf-8'.
Columnas en 'datos_casinos_salas.csv': ['Ruc', 'Empresa', 'Establecimiento', 'Giro', 'Resolución', 'Código Sala', 'Vigencia', 'Dirección', 'Distrito', 'Provincia', 'Departamento']
'resultados_geocodificados.csv' cargado exitosamente con codificación 'utf-8'.
Columnas en 'resultados_geocodificados.csv': ['Direccion_Original', 'Direccion_Exitosa', 'Latitud', 'Longitud', 'Estado', 'Geocodificador_Usado', 'Variantes_Probadas']
Eliminadas direcciones duplicadas en 'resultados_geocodificados.csv' para la columna 'Direccion_Original'.
Se eliminaron 0 filas completamente duplicadas después de la combinación.

Datos combinados guardados exitosamente en 'datos_casinos_geocodificados.csv' delimitado por punto y coma y codificación 'latin1'.
